In [ ]:
import pandas as pd
import igraph as ig
import random
#import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import matplotlib.pyplot as plt
import numpy as np

# Read correlation data from the Excel file
df = pd.read_excel('correlation_results.xlsx')

# Create an empty graph
g = ig.Graph(directed=False)

# Keep track of added vertices to avoid duplicates
added_vertices = set()

# Iterate over DataFrame rows to add edges with weights
for row in df[df['Edge'] == 'Yes'].itertuples(index=False):
    source = row[0]
    target = row[1]
    correlation = row[2]
    
    if source not in added_vertices:
        g.add_vertex(name=source)
        added_vertices.add(source)
    if target not in added_vertices:
        g.add_vertex(name=target)
        added_vertices.add(target)
    
    g.add_edge(source, target, weight=correlation)

In [ ]:
def ltm(graph, seed_set):
    activated_nodes = set(seed_set)  
    previous_activated_nodes = set() 


    while previous_activated_nodes != activated_nodes:
        previous_activated_nodes = set(activated_nodes)  

        # Iterate over unactivated nodes
        for vertex in graph.vs:
            if vertex.index not in activated_nodes:
                sum_neighbor_weights = 0
                all_neighbors = [neighbor.index for neighbor in vertex.neighbors()]  # Get indices of all neighbors      
                
                total_neighbor_weight = sum(graph.es[graph.get_eid(vertex.index, neighbor_index)]['weight'] for neighbor_index in all_neighbors)
                
                # Calculate sum of normalised edge weights connected to active nodes
                for neighbor_index in all_neighbors:
                    if neighbor_index in activated_nodes:
                        eid = graph.get_eid(vertex.index, neighbor_index)
                        edge_weight = graph.es[eid]['weight']
                        normalized_edge_weight = edge_weight / total_neighbor_weight if total_neighbor_weight != 0 else 0
                        sum_neighbor_weights += normalized_edge_weight
                        

                
                # Activate the node if the sum of normalised neighbor weights is >= 0.4
                if sum_neighbor_weights >= 0.4:
                    activated_nodes.add(vertex.index)

    spread = len(activated_nodes) - len(seed_set)  
    return spread,activated_nodes

def plot_network(graph, seed_set, activated_nodes):
    # Mapping node IDs to letters for visualisation
    id_to_letter = {i: chr(65 + i) for i in range(g.vcount())}
    vertex_labels = [f"{id_to_letter[id]}" for id in range(g.vcount())]

    layout = g.layout_fruchterman_reingold() 
    fig = make_subplots(rows=1, cols=1)

    for edge in g.get_edgelist():
        x0, y0 = layout[edge[0]]
        x1, y1 = layout[edge[1]]
        fig.add_scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(color='gray', width=1))

    fig.add_scatter(x=[coord[0] for coord in layout], y=[coord[1] for coord in layout],
                    mode='markers',
                    marker=dict(size=10, color=["blue" if v in seed_set else "lightblue" if v in activated_nodes else "black" for v in range(g.vcount())]),
                    text=vertex_labels, showlegend=False)

    fig.update_layout(
        width=1200, 
        height=800, 
        margin=dict(l=0, r=0, b=0, t=0),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    )
    fig.add_annotation(
        x=0,
        y=0,
        xref="paper",
        yref="paper",
        text=f"Number of Nodes: {g.vcount()}",
        showarrow=False,
        font=dict(size=14),
    )
    fig.show()



In [ ]:

def greedy_ltm(graph, k):
    seed_set = [] 
    spread = []    
    timelapse = [] 
    activated_nodes_sizes = []  
    start_time = time.time()  

    print("Starting greedy_ltm function")  

    for i in range(k):
        print(f"\nIteration {i + 1}:")
        best_spread = 0
        best_node = None
        
        for j in set(range(graph.vcount())) - set(seed_set):
            current_seed_set = seed_set + [j]
            current_spread, activated_nodes = ltm(graph, current_seed_set)  

            if current_spread is not None and current_spread > best_spread:
                best_spread = current_spread
                best_node = j
            
        if best_node is not None:
            seed_set.append(best_node)
            spread.append(best_spread)
            
            _,activated_nodes = ltm(graph,seed_set)  
            activated_nodes_sizes.append(len(activated_nodes)) 

            elapsed_time = time.time() - start_time
            timelapse.append(elapsed_time)  
            print("Current seed set:", seed_set)
            print("Best node:", best_node)
            print("Elapsed Time:", elapsed_time)
        else:
            print("No suitable node found, stopping iteration.")
            break

    print("Reached end of loop")   
    return seed_set, spread, timelapse, activated_nodes_sizes



k = 20
seed_set, spread, timelapse, activated_nodes_sizes = greedy_ltm(g, k)
print("seed set", seed_set)
_, activated_nodes = ltm(g, seed_set)
print(len(activated_nodes))
plot_network(g, seed_set, activated_nodes)

In [ ]:
def greedy_ltm2(graph, k):
    seed_set = [] 
    spread = []    
    timelapse = [] 
    activated_nodes_sizes = []  
    start_time = time.time()  

    print("Starting greedy_ltm2 function")  

    for i in range(int(k/2)):
        print(f"\n Iteration {i + 1}:")
        best_spread = 0
        best_nodes = None
        
        for j in set(range(graph.vcount())) - set(seed_set):
            for l in set(range(graph.vcount())) - set(seed_set):
                if l != j:  # Ensure the two nodes are distinct
                    current_seed_set = seed_set + [j, l]
                    current_spread, activated_nodes = ltm(graph, current_seed_set)  

                    if current_spread is not None and current_spread > best_spread:
                        best_spread = current_spread
                        best_nodes = (j, l)
                        

        if best_nodes is not None:
            seed_set.extend(best_nodes)
            spread.append(best_spread)
            
            _,activated_nodes = ltm(graph,seed_set) 
            activated_nodes_sizes.append(len(activated_nodes)) 
         
            elapsed_time = time.time() - start_time
            timelapse.append(elapsed_time)  
            print("Current seed set:", seed_set)
            print("Best nodes:", best_nodes)
            print("Elapsed Time:", elapsed_time)
        else:
            print("No suitable nodes found, stopping iteration.")
            break

    print("Reached end of loop")   
    return seed_set, spread, timelapse, activated_nodes_sizes


k=20
# Greedy function for selecting seed set of size 6 using greedy_ltm2
seed_set2, spread2, timelapse2, activated_nodes_sizes2 = greedy_ltm2(g, k)
_, activated_nodes2 = ltm(g, seed_set2)
# Plot the network
plot_network(g, seed_set2, activated_nodes2)


In [ ]:
def greedy_ltm2_dis(graph, k):
    seed_set = [] 
    spread = []    
    timelapse = [] 
    activated_nodes_sizes = []  # To store the number of activated nodes for each seed set size
    start_time = time.time()  

    print("Starting greedy_ltm2_dis function")  

    for i in range(int(k / 2)):
        print(f"\n Iteration {i + 1}:")
    
        best_spread = 0
        best_nodes = None
        
        for j in set(range(graph.vcount())) - set(seed_set):
            for l in set(range(graph.vcount())) - set(seed_set):
                # Ensure the two nodes are distinct and satisfy additional conditions
                if l != j and not any(graph.are_connected(j, neighbor) for neighbor in graph.neighbors(l)) \
                        and not any(graph.are_connected(l, neighbor) for neighbor in graph.neighbors(j)) \
                        and not graph.are_connected(j, l) and not graph.are_connected(l, j):

                    current_seed_set = seed_set + [j, l]
                    
                    #current_spread, _ = ltm(graph, current_seed_set)  # Get activated nodes
                    current_spread, activated_nodes = ltm(graph, current_seed_set)  # Get activated nodes

                    if current_spread is not None and current_spread > best_spread:
                        best_spread = current_spread
                        best_nodes = (j, l)
                        #activated_nodes_sizes.append(len(activated_nodes))

        if best_nodes is not None:
            seed_set.extend(best_nodes)
            spread.append(best_spread)
            
            _,activated_nodes = ltm(graph,seed_set) 
            activated_nodes_sizes.append(len(activated_nodes)) 
            elapsed_time = time.time() - start_time
            timelapse.append(elapsed_time) 
            print("Current seed set:", seed_set)
            print("Best nodes:", best_nodes)
            print("Elapsed Time:", elapsed_time)
        else:
            print("No suitable nodes found, stopping iteration.")
            break

    print("Reached end of loop")   
    return seed_set, spread, timelapse, activated_nodes_sizes

k=20
seed_set3, spread3, timelapse3, activated_nodes_sizes3 = greedy_ltm2_dis(g, k)
_, activated_nodes3 = ltm(g, seed_set3)
plot_network(g, seed_set3, activated_nodes3)

print(activated_nodes_sizes3)
print(timelapse3)


In [ ]:
def diffusion_degree(graph, vertex_index, threshold):
    vertex = graph.vs[vertex_index]
    # CDD(v)
    cd_v = graph.degree(vertex)
    # C'DD(v)
    cd_prime_v = threshold * cd_v
    # C''DD(v)
    c_double_prime_dd_v = 0
    
    # For each neighbor of the vertex
    for neighbor_index in graph.neighbors(vertex):
        neighbor = graph.vs[neighbor_index]

        # Calculate normalized edge weight (λ) from v to neighbor
        eid = graph.get_eid(vertex_index, neighbor_index)
        edge_weight = graph.es[eid]['weight']
        
        # Total weight of edges connected to the current neighbor
        total_neighbor_weight = sum(graph.es[graph.get_eid(neighbor.index, neighbor_neighbor.index)]['weight'] for neighbor_neighbor in neighbor.neighbors())
        
        #Normalise edge weight (λ) from v to neighbor
        normalized_edge_weight = edge_weight / total_neighbor_weight if total_neighbor_weight != 0 else 0
        
        # Calculate λ * (degree of neighbor - 1)
        lambda_times_degree_minus_1 = normalized_edge_weight * (graph.degree(neighbor) - 1)
        c_double_prime_dd_v += lambda_times_degree_minus_1

    # CDD(v) = C'DD(v)+ C''DD(v)
    cdd_v = cd_prime_v + c_double_prime_dd_v

    return cdd_v


def top_cdd(graph, threshold, k):
    diffusion_degrees = {}
    # For each vertex index in the graph
    for vertex_index in range(len(graph.vs)):
        diffusion_degrees[vertex_index] = diffusion_degree(graph, vertex_index, threshold)
  
    sorted_vertices = sorted(diffusion_degrees.items(), key=lambda x: x[1], reverse=True)
    
    top_cdd = [vertex_index for vertex_index, _ in sorted_vertices[:k]]
    
    return top_cdd




In [ ]:
def greedy_ltm(graph, k,top_cdd_nodes):
    seed_set = top_cdd_nodes
    spread = []    
    timelapse = [] 
    activated_nodes_sizes = []  
    start_time = time.time()  

    print("Starting greedy_ltm function")  

    for i in range(k):
        print(f"\n Iteration {i + 1}:")
        best_spread = 0
        best_node = None
        
        for j in set(range(graph.vcount())) - set(seed_set):
            current_seed_set = seed_set + [j]
            current_spread, activated_nodes = ltm(graph, current_seed_set) 

            if current_spread is not None and current_spread > best_spread:
                best_spread = current_spread
                best_node = j
            
        if best_node is not None:
            seed_set.append(best_node)
            spread.append(best_spread)
            
            _,activated_nodes = ltm(graph,seed_set)  
            activated_nodes_sizes.append(len(activated_nodes))  

            elapsed_time = time.time() - start_time
            timelapse.append(elapsed_time) 
            print("Current seed set:", seed_set)
            print("Best node:", best_node)
            print("Elapsed Time:", elapsed_time)
        else:
            print("No suitable node found, stopping iteration.")
            break

    print("Reached end of loop")   
    return seed_set, spread, timelapse, activated_nodes_sizes

import time

def top_cdd_time(graph, threshold, k):
    start_time = time.time()  
    
    diffusion_degrees = {}
    for vertex_index in range(len(graph.vs)):
        # Calculate diffusion degree for the current vertex
        diffusion_degrees[vertex_index] = diffusion_degree(graph, vertex_index, threshold)
    
    # Sort vertices by diffusion degree in descending order
    sorted_vertices = sorted(diffusion_degrees.items(), key=lambda x: x[1], reverse=True)
    
    # Select top k vertices with highest diffusion degree
    top_cdd = [vertex_index for vertex_index, _ in sorted_vertices[:k]]
    
    end_time = time.time()  
    elapsed_time = end_time - start_time  
    
    return top_cdd, elapsed_time


def greedy_ltm_with_top_cddratio(graph, k, ratio):
    seed_set = []
    spread = []    
    timelapse = [] 
    activated_nodes_sizes = []  
    start_time = time.time()  

    print("Starting greedy_ltm function")  
    threshold = 0.4
    top_cdd_nodes = top_cdd(graph, threshold, int(k * ratio))  # Get top nodes from top_cdd
    elapsed_time = time.time() - start_time
    print("Elapsed Time:", elapsed_time)

    # Add nodes from top_cdd to seed_set
    seed_set.extend(top_cdd_nodes)

    rest_nodes_count = k - len(top_cdd_nodes)  # Calculate the number of remaining nodes needed

    while rest_nodes_count > 0: 
        best_spread = 0
        best_unactivated_node = None
        
        # Find the best pair such that one is in top_cdd and the other is an unactivated node
        for i in top_cdd_nodes:
            for j in range(graph.vcount()):  
                if j not in top_cdd_nodes and j not in seed_set:  
                    current_seed_set = seed_set + [j]  # Add the unactivated node to the seed set
                    current_spread, _ = ltm(graph, current_seed_set)  

                    if current_spread is not None and current_spread > best_spread:
                        best_spread = current_spread
                        best_unactivated_node = j
        
        if best_unactivated_node is not None:
            seed_set.append(best_unactivated_node)
            spread.append(best_spread)
            elapsed_time = time.time() - start_time
            timelapse.append(elapsed_time)
            rest_nodes_count -= 1  
            print("\nCurrent seed set:", seed_set)
            print("Best unactivated node:", best_unactivated_node)
            print("Elapsed Time:", elapsed_time)
        else:
            print("No suitable unactivated node found, stopping iteration.")
            break
    
    print("Reached end of loop")   
    return seed_set, spread, timelapse, activated_nodes_sizes


ratio_values = [0.5,0.2, 0.4, 0.6, 0.8, 1]
k = 20

for ratio in ratio_values:
    print(f"\nTesting ratio: {ratio}, k: {k}")
    results = greedy_ltm_with_top_cddratio(g, k, ratio)
    seed_set, spread, timelapse, activated_nodes_sizes = results
    _, activated_nodes = ltm(g, seed_set)
    plot_network(g, seed_set, activated_nodes)



In [ ]:
#Greedy LTM
seed_set1 = [54, 83, 86, 91, 0, 1, 2, 3, 4, 5, 6, 7, 19, 34, 9, 8, 18, 10, 13, 11]
#Greedy LTM 2
seed_set2= [54, 83, 86, 91, 7, 19, 34, 48, 0, 1, 8, 18, 69, 72, 2, 3, 6, 11, 13, 14]
#Greedy LTM 2 with distance 
seed_set3= [54, 86, 83, 91, 0, 80, 1, 88, 5, 82, 6, 69, 17, 90, 19, 79, 29, 51, 34, 78]
#Greedy with 0.8 centrality
seed_set4=[2, 18, 13, 49, 47, 10, 14, 3, 27, 1, 24, 7, 25, 8, 28, 36, 86, 91, 0, 4]

#Greedy LTM
corresponding_names_1 = []
for vertex_index in seed_set1:
    vertex_name = g.vs[vertex_index]['name']
    corresponding_names_1.append(vertex_name)
print("Greedy LTM: \n",corresponding_names_1)


# Greedy LTM 2
corresponding_names_2 = []
for vertex_index in seed_set2:
    vertex_name = g.vs[vertex_index]['name']
    corresponding_names_2.append(vertex_name)
print("\nGreedy LTM 2: \n", corresponding_names_2)

# Greedy LTM 2 with distance
corresponding_names_3 = []
for vertex_index in seed_set3:
    vertex_name = g.vs[vertex_index]['name']
    corresponding_names_3.append(vertex_name)
print("\nGreedy LTM 2 with distance: \n", corresponding_names_3)

# Greedy with 0.5 centrality
corresponding_names_4 = []
for vertex_index in seed_set4:
    vertex_name = g.vs[vertex_index]['name']
    corresponding_names_4.append(vertex_name)
print("\nGreedy with 0.5 centrality: \n", corresponding_names_4)

names_set1 = set(corresponding_names_1)
names_set2 = set(corresponding_names_2)
names_set3 = set(corresponding_names_3)
names_set4 = set(corresponding_names_4)

# Find the common names among all sets
common_names = names_set1.intersection(names_set1,names_set2, names_set3, names_set4)

# Print the common names
print("\nStocks in all seed sets:", common_names)


# Print the degree of each stock in common_names
for stock in common_names:
    # Find the index of the stock in the graph
    stock_index = g.vs.find(name=stock).index
    
    # Get the degree of the stock using the index
    stock_degree = g.degree(stock_index)
    
    # Print the stock name and its degree
    print(f"Stock: {stock}, Degree: {stock_degree}")
